In [1]:
#구글 드라이브 연결

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 내 구글 드라이브로 이동
%cd "/content/drive/MyDrive"

%cd '/content/'

!git clone https://github.com/ultralytics/yolov5.git

!pip install -U -r yolov5/requirements.txt

In [ ]:
import torch

#파이토치 버전 확인, cuda device properties 확인
print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 1.13.0+cu117 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
#오픈cv 버전 확인

import cv2
print(cv2.__version__)

4.6.0


In [ ]:
#사진 가져오기

!mkdir datasets
%cd /content/datasets/

!curl -L "https://universe.roboflow.com/ds/Gega6ATJkJ?key=2YG8UuuPNf" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
#test, train, valid 나누는 명령어 쓰기 위한 거였는데
#요즘 파일은 나눠져서 나오는듯
#그러면 10번코드 안쓰고 여기서 train, valid 리스트 만들어서
#txt 만들어도 될듯?

%cd /

from glob import glob

train_img_list = glob('/content/datasets/train/images/*.jpg')
val_img_list = glob('/content/datasets/valid/images/*.jpg')

print(len(train_img_list))
print(len(val_img_list))

/
2616
335


In [ ]:
# from sklearn.model_selection import train_test_split

# train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

2616 335


In [ ]:
with open('/content/datasets/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')
with open('/content/datasets/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')  

In [ ]:
import yaml

with open('/content/datasets/data.yaml', 'r') as f:
  #data = yaml.load(f) 버전 바뀌고 명령어 바뀜
  data = yaml.full_load(f)
print(data)

data['train'] = '/content/datasets/train.txt'
data['val'] = '/content/datasets/val.txt'

with open('/content/datasets/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 1, 'names': ['Kids']}
{'train': '/content/datasets/train.txt', 'val': '/content/datasets/val.txt', 'nc': 1, 'names': ['Kids']}


In [ ]:
#학습 마지막 이름 바꾸기

%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /content/datasets/data.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name kids_yolov5s_results

/content/yolov5
train: weights=yolov5s.pt, cfg=/content/yolov5/models/yolov5s.yaml, data=/content/datasets/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=kids_yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-235-g75728bb Python-3.7.15 torch-1.13.0+cu117 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, ob

In [ ]:
#드라이브저장

!cp -r /content/yolov5/runs/train/kids_yolov5s_results /content/drive/MyDrive/models

In [ ]:


%cd /content/yolov5/

!python detect.py --source /content/drive/MyDrive/kids/test3.mp4 --weights /content/drive/MyDrive/models/weights/best.pt

/content/yolov5
detect: weights=['/content/drive/MyDrive/models/weights/best.pt'], source=/content/drive/MyDrive/kids/test3.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-235-g75728bb Python-3.7.15 torch-1.13.0+cu117 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/484) /content/drive/MyDrive/kids/test3.mp4: 384x640 3 Kidss, 19.0ms
video 1/1 (2/484) /content/drive/MyDrive/kids/test3.mp4: 384x640 3 Kidss, 8.9ms
video 1/1 (3/484) /content/drive/MyDrive/kids/test3.mp4: 384x640 3 Kidss, 10.2ms
video 1/1 (4/484) /content/drive/MyDrive/kids/test3.m